<a href="https://colab.research.google.com/github/GustavoFelips/GoogleColab/blob/main/Processamento_de_Linguagem_Natural_IAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dataset Amazon Reviews

In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Input
from sklearn.metrics import classification_report
import numpy as np
from tensorflow.keras.utils import to_categorical

# Download e descompactação da base de dados
import os
import tarfile
import urllib.request


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kritanjalijain/amazon-reviews")

print("Path to dataset files:", path)

100%|██████████| 1.29G/1.29G [00:06<00:00, 225MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kritanjalijain/amazon-reviews/versions/2


Diretório onde o dataset foi baixado

In [3]:
import os

pasta = '/root/.cache/kagglehub/datasets/kritanjalijain/amazon-reviews/versions/2'

# Listar todos os arquivos e pastas
itens = os.listdir(pasta)

# Exibir os itens
for item in itens:
    print(item)


test.csv
train.csv
amazon_review_polarity_csv.tgz


In [4]:
## Carregando os dados de treino e teste
train_dir = os.path.join(path, 'train.csv')
test_dir = os.path.join(path, 'test.csv')

In [5]:
import numpy as np
import pandas as pd

df_train = pd.read_csv(train_dir,names=['polarity', 'title ', 'text'])
df_test = pd.read_csv(test_dir,names=['polarity', 'title ', 'text'])

#polarity - 1 for negative and 2 for positive



In [6]:
df_test.head()

,polarity,title,text
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...


In [7]:
x_train, y_train = np.array(df_train['text']), np.array(df_train['polarity'])
x_test, y_test = np.array(df_test['text']), np.array(df_test['polarity'])

In [8]:
# Definindo o número máximo de palavras a serem consideradas
num_words = 1000

# Instanciando o tokenizador
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(x_train)

In [9]:
# Convertendo as reviews para sequências de números inteiros
X_train = tokenizer.texts_to_sequences(x_train)
X_test = tokenizer.texts_to_sequences(x_test)

In [10]:
# Aplicando padding para padronizar o tamanho das sequências
maxlen = 400
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)


In [11]:
# Convertendo as labels para one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
# Exemplo de uma review tokenizada e padronizada
print("Review tokenizada e padronizada:", X_train[0])

Review tokenizada e padronizada: [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  

In [13]:
# Definindo a arquitetura do modelo
model = Sequential([
    Input(shape=(maxlen,)),  # Camada de entrada
    Embedding(input_dim=num_words, output_dim=32),  # Embedding
    SimpleRNN(32),  # RNN simples
    Dense(3, activation='softmax')  # Camada de saída
])

In [14]:
# Compilação do modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
# Exibindo o resumo do modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 32)           32000     
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 3)                 99        
                                                                 
Total params: 34179 (133.51 KB)
Trainable params: 34179 (133.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Treinamento do modelo
history = model.fit(X_train, y_train, epochs=5, batch_size=521)


Epoch 1/5
6910/6910 [==============================] - 861s 124ms/step - loss: 0.3314 - accuracy: 0.8586
Epoch 2/5
6910/6910 [==============================] - 861s 125ms/step - loss: 0.2665 - accuracy: 0.8903
Epoch 3/5
6910/6910 [==============================] - 850s 123ms/step - loss: 0.2535 - accuracy: 0.8967
Epoch 4/5
6910/6910 [==============================] - 847s 123ms/step - loss: 0.2459 - accuracy: 0.9003
Epoch 5/5
6910/6910 [==============================] - 861s 125ms/step - loss: 0.2415 - accuracy: 0.9025


In [17]:
# Avaliação no conjunto de teste
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Acurácia no conjunto de teste: {test_acc:.4f}")

12500/12500 [==============================] - 229s 18ms/step - loss: 0.2389 - accuracy: 0.9041
Acurácia no conjunto de teste: 0.9041


In [18]:
# Predição de uma amostra do conjunto de teste
sample_idx = 24
sample_review = X_test[sample_idx].reshape(1, -1)  # Reshape para ajustar ao input do modelo

# O modelo retorna probabilidades para cada classe (negativa e positiva)
predicted_probabilities = model.predict(sample_review)

# A classe prevista será a que tiver a maior probabilidade
predicted_class = np.argmax(predicted_probabilities, axis=1)

# O rótulo real também precisa ser convertido de one-hot para o índice da classe
real_class = np.argmax(y_test[sample_idx])

# Exibir a review, classe prevista e classe real
print(f"Review: {X_test[sample_idx]}")
print(f"Classe prevista: {'Positiva' if predicted_class == 2 else 'Negativa'}")
print(f"Classe real: {'Positiva' if real_class == 2 else 'Negativa'}")


1/1 [==============================] - 0s 169ms/step
Resenha: [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  